# Proceso GLUE Eventos Amplitude

## 1. Cargamos las librerias

In [ ]:


#--------------------------------------------------------------------------------------------------------------

# TIPOS DE TRANSACCIONES
print('3. PARAMETRIA DE TRANSACCIONES')
plan_user_to_user = '18-C2C LOCAL'
commerce_automatic_debit = ['00300008', '00300009', '00300007', '00300000', '00300001', '00300006', '00300002',
                            '00300005', '00300004']
plan_automatic_debit = ['15-DEBITO AUTOMATI', '16-DEBITO AUTOMATI']
commerce_telerecargas = ['00300000']
commerce_investments_withdraw = ['00300009']
commerce_bill_payment = ['00300001']
commerce_cash_out_cvu = ['00300007']
plan_withdraw_atm = '3-ADELANTOS EN PES'
commerce_loan_installment = ['00300002']


transaction_type_rfm=['USER_TO_USER','WITHDRAW_ATM','AUTOMATIC_DEBIT','TELERECARGAS_CARGA','CONSUMPTION_POS',
                      'LOAN_INSTALLMENT_PAYMENT','BILL_PAYMENT', 'CASH_OUT_CVU','INVESTMENTS_DEPOSIT']

#transaction_type_rfm=['USER_TO_USER','AUTOMATIC_DEBIT','TELERECARGAS_CARGA','CONSUMPTION_POS','BILL_PAYMENT',
#                      'CASH_OUT_CVU','WITHDRAW_ATM','INVESTMENTS_DEPOSIT']



#---------------------------- OBTENER LAS COTIZACIONES DE DOLAR  ------------------------------------
print('NOW:',datetime.now())

#----------------------------- COTIZACIONES --------------------------------------
print('4. IMPORTAR EL ARCHIVO DE COTIZACIONES, TRANSFORMAR')

cotizaciones_key= 'parameters'
cotizaciones_fn = 'cotizaciones_stage.csv'
cotizaciones_path = f's3://{recommendations_bucket}/{cotizaciones_key}/{cotizaciones_fn}'
coti = pd.read_csv(cotizaciones_path, usecols=[1,5])
coti['fecha'] = pd.to_datetime(coti['fecha'])
coti['fecha_mes'] = coti['fecha'].dt.to_period('M')
coti_gb = coti.groupby('fecha_mes', as_index=False).agg({'venta_uala':'max'})
df_cotizaciones = coti.merge(coti_gb, left_on='fecha_mes', 
                             right_on='fecha_mes', 
                             how='left', suffixes=('', '_max_mes'))[['fecha','venta_uala_max_mes']]

path_csv = f's3://{recommendations_bucket}/{cotizaciones_key}/cotizacion_mensual.csv'
df_cotizaciones.to_csv(path_or_buf=path_csv)
    
# IMPORTAR EL ARCHIVO DE LAS COTIZACIONES 
valor_cotizacion= (spark.read
                   .option('header',True)
                   .option("inferSchema","true")
                   .csv(path_csv)
                  )

cot_spark = valor_cotizacion.select(['fecha', 'venta_uala_max_mes'])
cot_spark = (cot_spark
             .withColumn('fecha', F.to_date(cot_spark.fecha,"yyyy-MM-dd"))
             .withColumnRenamed('fecha', 'fecha_venta')
             .withColumn('venta_uala_max_mes', cot_spark.venta_uala_max_mes.cast('double'))
            )

#----------------------------- SURVIVAL ACCOUNTS ------------------------------------
# Ir a buscar accounts que usa survival para usar las mismas
print('5. OBTENER SURVIVAL ACCOUNTS')
keys_surv_accounts='data/raw/users'

files_objets_surv, ultima_fecha_surv = list_objects_function(last_day,first_day,buckets_ = survival_bucket,keys_ = keys_surv_accounts, retrieve_last=True)

print(f'Hay {len(files_objets_surv)} archivos de survival en la carpeta más reciente datada en {ultima_fecha_surv}')
df_survival = spark.read.parquet(*files_objets_surv).select(['accountgp'])


#--------------------------------------------------------------------------------------------------------------

# LEER ARCHIVOS 7001
print('6. LEER ARCHIVOS 7001')

columns = ['fechaautorizacion','cuenta', 'plan', 'comercio', 'moneda', 'estado','importe','reversa']
files7001=list_objects_function(last_day=last_day,first_day=first_day,buckets_=bucket_amplitude_data,keys_=path_key_amplitude)

# to read parquet file
print('La cantidad de parquets 7001 es', len(files7001))

df = spark.read.parquet(*files7001).select(columns)
print('La cantidad de filas es:',df.count(),'y la cantidad columnas', len(df.columns))


#----------------------------- TRANSFORMACIONES 7001 --------------------------------------
print('7. TRANSFORMACIONES 7001')

# CONVERTIR EN FORMATO FECHA
print('7.1. FORMATO FECHA')
df=df.withColumn("fecha",F.substring(df['fechaautorizacion'], 1, 8))
df=df.drop('fechaautorizacion')
#original
#df=df.withColumn('fecha',F.to_date(df.fecha,"ddmmyyyy"))
#propuesta
#df=df.withColumn('fecha',F.unix_timestamp(df.fecha,"ddMMyyyy"))
df=df.withColumn('fecha',F.to_date(df.fecha,"ddMMyyyy"))


# FORMATO CUENTA, IMPORTE
print('7.2. FORMATO IMPORTE')

df = (df
        .withColumn("cuenta", df["cuenta"].cast('integer'))
        .withColumn('importe', F.regexp_replace('importe', '-', ''))
     )
df = df.withColumn("importe", df["importe"].cast('double'))


###  FILTER SURVIVAL !!!  ####
print('7.3. INNER JOIN SURVIVAL')
df_survival = df_survival.withColumn("accountgp", df_survival["accountgp"].cast('integer'))
df_survival = df_survival.distinct()
#df_survival = df_survival.dropDuplicates()
df = df.join(df_survival, df.cuenta == df_survival.accountgp, how = 'inner')
########


# ELIMINO LAS TRANSACCIONES CANCELADAS
print('7.4. ELIMINO LAS TRANSACCIONES CANCELADAS DEL ARCHIVO 7001')

df = df.where((df.estado == 'APROBADA') & (~df.plan.isin(['5-DEVOLUCION PESOS', '6-DEVOLUCION DOLAR'])))
df = df.withColumn("cuenta", df["cuenta"].cast('integer'))
print('La cantidad de filas es:',df.count(),'y la cantidad columnas', len(df.columns))



### JOIN PARA TRAER COTIZACIONES
print('7.5. JOIN PARA TRAER COTIZACIONES')

df = df.join(cot_spark, df.fecha == cot_spark.fecha_venta, how = 'left')
print('Cantidad de fechas sin cotizacion: ',df.where(F.col('venta_uala_max_mes').isNull()).select('venta_uala_max_mes').count())

# FORMATO Y FILTRADO IMPORTE 
print('7.6. FORMATO, FILTRADO, IMPORTE Y CONVERSION USD')

df=(
    df
        .withColumn("importe", df["importe"]/100)
    # FORMAT: REVERSA
        .withColumn("reversa", df["reversa"].cast('integer'))
    # CONVERTIR EL IMPORTE SI ES UNA EN REVERSA
        .withColumn('importe', F.when(F.col('reversa') == 1, 
                                      (F.col('importe')*-1)).otherwise(F.col('importe')))
    # PASAR A DOLAR
        .withColumn('importe_dolar', F.when(F.col('moneda') == '002', 
                                            (F.col('importe'))).otherwise(F.col('importe')/F.col('venta_uala_max_mes')))
)        

# FILTRO DE TRX CON IMPORTE DOLAR > 1 
df = df.filter(df['importe_dolar'] > 1)


# ELIMINAR COLUMNAS
df=df.drop('importe','reversa', 'fecha_venta')


# DEFINIR UNA VARIABLE TRANSACTION_TYPE
print('7.7 7001 DEFINIR EL TRANSCTIONTYPE')


df=df.withColumn("transaction_type", F.lit('CONSUMPTION_POS'))

# USER TO USER
df=df.withColumn('transaction_type', 
                    F.when(F.col('plan') == plan_user_to_user, 
                          'USER_TO_USER').otherwise(df['transaction_type']))

# AUTOMATIC DEBIT
df=df.withColumn('transaction_type', 
                     F.when((~F.col('comercio').isin(plan_automatic_debit)) & 
                          ( F.col('plan').isin(plan_automatic_debit)), 
                          'AUTOMATIC_DEBIT').otherwise(df['transaction_type']))

# TELERECARGAS CARGA
df=df.withColumn('transaction_type', 
                     F.when(F.col('comercio').isin(commerce_telerecargas), 
                          'TELERECARGAS_CARGA').otherwise(df['transaction_type']))

# INVESTMENTS WITHDRAW
df=df.withColumn('transaction_type', 
                     F.when(F.col('comercio').isin(commerce_investments_withdraw), 
                          'INVESTMENTS_WITHDRAW').otherwise(df['transaction_type']))

# CASH OUT CVU
df=df.withColumn('transaction_type', 
                     F.when(F.col('comercio').isin(commerce_cash_out_cvu), 
                          'CASH_OUT_CVU').otherwise(df['transaction_type']))

# WITHDRAW ATM
df=df.withColumn('transaction_type', 
                     F.when(F.col('plan').isin(plan_withdraw_atm), 
                          'WITHDRAW_ATM').otherwise(df['transaction_type']))

# BILL PAYMENTS
df=df.withColumn('transaction_type', 
                     F.when(F.col('comercio').isin(commerce_bill_payment), 
                          'BILL_PAYMENT').otherwise(df['transaction_type']))

# LOAN INSTALLMENT PAYMENT
df=df.withColumn('transaction_type', 
                     F.when(F.col('comercio').isin(commerce_loan_installment), 
                          'LOAN_INSTALLMENT_PAYMENT').otherwise(df['transaction_type']))


# FILTRAR POR LAS TRANSACCIONES
df=df.filter(df.transaction_type.isin(transaction_type_rfm))


fecha_min = df.agg({"fecha": "min"}).collect()[0]
fecha_max = df.agg({"fecha": "max"}).collect()[0]
print(f'Luego de todos los filtros hay {df.count()} filas')
print(f'Fecha más vieja es {fecha_min}')
print(f'Fecha más nueva es {fecha_max}')



#########################
#     Groupby total     #
#########################
print('8. GROUP BY TOTAL')


df = (df    
      .groupBy(['cuenta', 'fecha'])
      .agg(F.sum('importe_dolar').alias('importe_dolar')
     ))

base_ =(df
        .groupBy('cuenta')
        .agg(F.expr('count(distinct fecha)-1').alias('frequency'),
            F.datediff(F.max('fecha'), F.min('fecha')).alias('recency'),
            F.datediff(F.lit(last_day), F.min('fecha')).alias('T'),
            F.mean('importe_dolar').alias('importe_dolar')
            ))


base_ = base_.filter(base_['frequency'] > 0)


#########################
# Separar cal y holdout #
#########################
print('9. SEPARACION CAL Y HOLDOUT')

### CAL ###  <last_day-30
print('9.1 CAL')


cal_date=(last_day-pd.offsets.DateOffset(days=30)).date()

print(f'cal date es {cal_date}')

df_cal = df.where(F.col('fecha') < cal_date)

base_cal=(df_cal
   .groupBy('cuenta')
   .agg(F.expr('count(distinct fecha)-1').alias('frequency_cal'),
        F.datediff(F.max('fecha'), F.min('fecha')).alias('recency_cal'),
        F.datediff(F.lit(last_day), F.min('fecha')).alias('T_cal'),
        F.mean('importe_dolar').alias('importe_dolar_cal')
       ))


### HOLDOUT ### >=last_day-30
print('9.2 HOLDOUT')

df_holdout = df.filter(F.col('fecha').between(cal_date, last_day))

base_holdout=(df_holdout
       .groupBy('cuenta')
       .agg(F.expr('count(distinct fecha)-1').alias('frequency_holdout'),
            F.datediff(F.max('fecha'), F.min('fecha')).alias('recency_holdout'),
            F.datediff(F.lit(last_day), F.min('fecha')).alias('T_holdout'),
            F.mean('importe_dolar').alias('importe_dolar_holdout')
       ))

base_holdout = base_holdout.withColumnRenamed('cuenta', 'cuenta_holdout')

####### JOIN holdout-cal #######
print('10. JOIN HOLDOUT-CAL')


base_train = base_cal.join(base_holdout, base_cal.cuenta == base_holdout.cuenta_holdout, how = 'outer')
base_train = base_train.withColumn('duration_holdout', F.lit('30'))
base_train = base_train.withColumn('cuenta', F.coalesce(base_train.cuenta, base_train.cuenta_holdout)) 
base_train = base_train.drop('cuenta_holdout')
base_train = base_train.withColumn('duration_holdout', F.lit('30'))

base_train = base_train.withColumn('frequency_cal', F.coalesce(base_train.frequency_cal, F.lit("0") )) 
base_train = base_train.filter(base_train['frequency_cal'] > 0)

################################
print('11. ESCRIBIR PARQUET BASE Y BASE_TRAIN')

base_.write\
     .format('parquet')\
     .save(f's3://{recommendations_bucket}/data/dt={str(last_day)}/full', mode='overwrite')

base_train.write\
     .format('parquet')\
     .save(f's3://{recommendations_bucket}/data/dt={str(last_day)}/train', mode='overwrite')



print('Ubicación files', f's3://{recommendations_bucket}/data/dt={str(last_day)}')
print('Eliminar archivos vacíos de Spark')
print('Función retrieve files')

def retrieve_files(path, file_type, list_dates):
    bucket=path.split('/')[2]
    prefix='/'.join(path.split('/')[3:])
    list_objects=list(s3.Bucket(bucket).objects.all())
    list_objects=[f's3://{bucket}/{i.key}' for i in list_objects if ((i.key.find(prefix)>=0) & any(x in i.key.lower() for x in list_dates) & (i.key.find(file_type)>=0))]
    return list_objects



delete_files = retrieve_files(path=f's3://{recommendations_bucket}/data/', file_type='$folder$', list_dates=['$folder$'])
print('Files to delete', delete_files)
files_keys=[]
for i in range(0,len(delete_files)):
    files_keys=files_keys+[{'Key':('/').join(delete_files[i].split('/')[3:])}]
if len(files_keys)>0:
    s3_client.delete_objects(Bucket=recommendations_bucket,
                             Delete={'Objects':files_keys})
del delete_files
gc.collect()

                              
print('NOW:', datetime.now())  

In [1]:
!pip install -q awswrangler

In [16]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
import dask.array as da
import boto3
import ast
from datetime import datetime, timedelta
import awswrangler as wr
from itertools import chain
import gc
import sys
import time
from sklearn import preprocessing

glue = boto3.client('glue')
s3 = boto3.resource('s3')
ssm = boto3.client('ssm') 
lakeformation = boto3.client('lakeformation')

### 2. Armamos el proceso en "GLUE LTV-RFM AR"

In [72]:
%%writefile get_data_amplitude.py

import sys
import pyspark.sql.functions as func
from awsglue.dynamicframe import DynamicFrame
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.sql.types import *
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
import pyspark.sql.functions as F
import json
import boto3
import ast
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import gc
import sys
from pyspark.conf import SparkConf
import pandas as pd

print('Lectura de parámetros')

# ----------------------------------------------------------------------------------
print('NOW:', datetime.now())

args = getResolvedOptions(sys.argv,
                          ['bucket_amplitude_data', 
                           'today', 
                           'kms_key_arn', 
                           'recommendations_bucket'])

bucket_amplitude_data = args['bucket_amplitude_data']
recommendations_bucket = args['recommendations_bucket']
kms_key_id = args['kms_key_arn']
today = args['today']


#--------------------------------------------------------------------------------------------------------------

print('Spark Configuración')

spark_conf = SparkConf().setAll([
  ("spark.hadoop.fs.s3.enableServerSideEncryption", "true"),
  ("spark.hadoop.fs.s3.serverSideEncryption.kms.keyId", kms_key_id)
])

sc = SparkContext(conf=spark_conf)
glueContext = GlueContext(sc)
spark = glueContext.spark_session
logger = glueContext.get_logger()



print('Crear objetos S3-ssm')
# ----------------------------------------------------------------------------------
s3 = boto3.resource('s3')
s3_client = boto3.client('s3')
ssm = boto3.client('ssm')

#--------------------------------------------------------------------------------------------------------------
print('Parámetros:')
path_key_amplitude = 'ar/amplitude/tb_ar_amplitude_events_stage/'

## FECHAS INTERVALO
#print('1. CALCULO DE FECHAS')
##Today llevado al primero del mes menos 1 día
#today = datetime.strptime(today, '%Y-%m-%d').date().replace(day=1)
#last_day=(today-pd.offsets.DateOffset(days=1)).date()
##
#first_day=(last_day-pd.offsets.DateOffset(days=365)).date()
#
#print('2. Intevalo de fechas analizada: ',first_day,'y',last_day)

def first_and_last(today):
    fecha=datetime.strptime(today, '%Y-%m-%d').date()
    first_day=fecha.replace(day=1)
    next_month = fecha.replace(day=28) + timedelta(days=4)
    last_day_of_month = next_month - timedelta(days=next_month.day)
    return first_day,last_day_of_month

print('Declaración de funciones')
def list_objects_function(buckets_, first_day, last_day, keys_, retrieve_last=False):
    
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(buckets_)
    files_in_bucket = list(bucket.objects.all())
    files_objets = [f"s3://{buckets_}/" + i.key for i in files_in_bucket if
                        (i.key.find(keys_) >= 0) and (i.key.find('.parquet') >= 0)]
    df_bucket_files = pd.DataFrame({
            'key': [i[:(i.find('dt=') + 14)] for i in files_objets],
            'path': files_objets,
            'date': pd.to_datetime([i[(i.find('dt=') + 3):(i.find('dt=') + 13)] for i in files_objets])
        })
    files=list(df_bucket_files.loc[df_bucket_files['date'].between(str(first_day),str(last_day)),'path'].values)
    return files



map_events = {
    "cuoti_selecciona_elegircuotas" : "cuotificaciones",
    "cuoti_sigue_seleccion_consumos" : "cuotificaciones",
    "prestamos_selecciona_simular_prestamo": "prestamos",
    "prestamos_espera": "prestamos",
    "general_ingresa_promociones": "promociones",
    "recargas_click_empezar": "recargas",
    "recargas_click_repetir": "recargas",
    "transferencia_selecciona_tieneuala": "transferencia_c2c",
    "transferencia_selecciona_notieneuala": "transferencia_cvu",
    "general_ingresa_cobros": "cobros",
    "cobros_acepta_tyc" : "cobros",
    "cobros_elige_link": "cobros",
    "cobros_elige_mpos": "cobros",
    "pagos_empezar": "pago_servicios",
    "click_inversiones":"inversiones"
}

eventos_recommendations = list(map_events.keys())
#-----------------------------------------------------------------------------------------------------------------
first_day,last_day = first_and_last(today)
print('Primer dia',first_day)
print('Ultimo dia',last_day)

files_objets_amplitude = list_objects_function(bucket_amplitude_data, first_day, last_day ,path_key_amplitude)

print(f'Hay {len(files_objets_amplitude)} archivos de survival en la carpeta')
df_amplitude = spark.read.parquet(*files_objets_amplitude).select(['user_id',"os_name","event_type","event_time"])
df_amplitude=df_amplitude.filter(df_amplitude.event_type.isin(eventos_recommendations))

df_amplitude = df_amplitude.withColumn('year_month', F.date_format(df_amplitude.event_time,'YYYY-MM'))

df_amplitude = df_amplitude.drop("event_time")

df_amplitude = df_amplitude.na.replace(map_events,1,"event_type")

df_amplitude = (df_amplitude    
      .groupBy(['user_id', 'event_type', 'year_month'])
      .agg(F.count('event_type').alias('cant'),
           F.max('os_name').alias('os_name'))
      .groupBy(['user_id','year_month','os_name'])
      .pivot("event_type")
      .agg(F.sum('cant'))
      .na.fill(0)
      )

df_amplitude.write\
     .format('parquet')\
     .save(f's3://{recommendations_bucket}/data/raw/amplitude/dt={str(first_day)}', mode='overwrite')

print('Ubicación files', f's3://{recommendations_bucket}/data/raw/amplitude/dt={str(first_day)}')

#DELETE $FOLDER$

def retrieve_files(path, file_type, list_dates):
    bucket=path.split('/')[2]
    prefix='/'.join(path.split('/')[3:])
    list_objects=list(s3.Bucket(bucket).objects.all())
    list_objects=[f's3://{bucket}/{i.key}' for i in list_objects if ((i.key.find(prefix)>=0) & any(x in i.key.lower() for x in list_dates) & (i.key.find(file_type)>=0))]
    return list_objects


delete_files = retrieve_files(path=f's3://{recommendations_bucket}/data/', file_type='$folder$', list_dates=['$folder$'])
print('Files to delete', delete_files)
files_keys=[]
for i in range(0,len(delete_files)):
    files_keys=files_keys+[{'Key':('/').join(delete_files[i].split('/')[3:])}]
if len(files_keys)>0:
    s3_client.delete_objects(Bucket=recommendations_bucket,
                             Delete={'Objects':files_keys})
del delete_files
gc.collect()

print(df_amplitude.show())
print((df_amplitude.count(), len(df_amplitude.columns)))

Overwriting get_data_amplitude.py


In [73]:
job_name='test-job_recommendations_amplitude'

In [74]:
# borrar job
#glue.delete_job(
#    JobName=job_name
#)

## 3. Generamos los parametros

In [75]:
today = '2021-01-10'
bucket_amplitude_data='churn-ds-stage'  ## AFIP, GP, etc
recommendations_bucket='test-uala-arg-datalake-aiml-recommendations'  # Para outputs
kms_key_arn='arn:aws:kms:us-east-1:322149183112:key/9cc44b23-c5e9-46cb-9987-0982d21f8d00' ## key para desencriptar

In [76]:
s3 = boto3.resource('s3')

# Guardar el archivo .py
s3.meta.client.upload_file('get_data_amplitude.py', 
                           recommendations_bucket, #bucket
                           'artifacts/code/amplitude/get_data_amplitude.py' #key+filename
)
print('.py uploaded')

.py uploaded


## 4. Creamos el job de GLUE

In [77]:
#job = glue.create_job(Name=job_name, 
#                      GlueVersion='2.0',
#                      Role='ML_AWSGlueServiceRole',
#                      Command={'Name': 'glueetl',
#                               'ScriptLocation': f's3://{recommendations_bucket}/artifacts/code/amplitude/get_data_amplitude.py'},
#                      DefaultArguments={
#                        '--additional-python-modules': 'dateutil==2.8.1'},
#                      MaxCapacity=3
#                      )

In [78]:
job_run = glue.start_job_run(
    JobName = job_name,
    Arguments = {
        '--today':today,
        '--bucket_amplitude_data': bucket_amplitude_data,
        '--recommendations_bucket': recommendations_bucket,
        '--kms_key_arn': kms_key_arn
    } 
)

In [79]:
print(job_run)

{'JobRunId': 'jr_cf2978c1ed6c989cac356896b755adc916606f4449ff8bf16a36c86932e7beca', 'ResponseMetadata': {'RequestId': 'ded488e5-e827-4452-bb2d-5e053957841f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 20 May 2021 19:56:31 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '82', 'connection': 'keep-alive', 'x-amzn-requestid': 'ded488e5-e827-4452-bb2d-5e053957841f'}, 'RetryAttempts': 0}}


In [80]:
MAX_WAIT_TIME=time.time() + 60*10 # 1 hour
max_time = time.time() + MAX_WAIT_TIME
while time.time() < max_time:
    response=glue.get_job_run(JobName=job_name, RunId=job_run['JobRunId'])
    status = response['JobRun']['JobRunState']
    print('Job run: {}'.format(status))
    
    if status == 'SUCCEEDED' or status == 'FAILED':
        break
        
    time.sleep(45)

Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: SUCCEEDED


## 5. Controlamos la carga de datos en el bucket


In [60]:
df=wr.s3.read_parquet(f's3://test-uala-arg-datalake-aiml-recommendations/data/raw/amplitude',dataset=True,partition_filter=lambda x: '2021-01-01' in x["dt"])

In [62]:
df.head(5)

,user_id,year_month,os_name,cobros,cuotificaciones,inversiones,pago_servicios,prestamos,promociones,recargas,transferencia_c2c,transferencia_cvu,dt
0,00360d0c-2638-458b-a1b6-144cb3ba4b78,2021-01,ios,0,0,4,0,0,0,0,4,1,2021-01-01
1,00509604-ce55-47db-93f6-6bca3867ef05,2021-01,android,0,0,0,0,0,0,0,10,4,2021-01-01
2,005e8359-5408-4b28-b2a0-a5ef6955f158,2021-01,android,0,0,0,0,0,0,0,2,1,2021-01-01
3,0072cfda-2213-4b1a-b4bf-3969ccacde3f,2021-01,android,2,0,0,0,0,0,4,4,1,2021-01-01
4,0074aa89-9148-4d94-9bee-e55bd591d038,2021-01,android,0,2,13,2,0,0,5,10,4,2021-01-01
